# ACS/WFC Reduction Tutorial

### Tyler D. Desjardins<br>March 2018


# 1. Introduction

This notebook covers the steps necessary to calibrate Advanced Camera for Surveys (ACS) Wide Field Channel (WFC) observations to produce a distortion-corrected image ready for photometry. Most of the information contained in this example can also be used for other *HST* instruments with minor adjustments. For more information, please see the documentation for your instrument of choice. A brief description of the ACS instrument follows. More details may be found on the [ACS website]() and in the [ACS Instrument]() and [Data Handbooks]().

ACS is a third generation *HST* instrument that was installed during Servicing Mission 3B in March, 2002. The instrument is comprised of three cameras: the already mentioned WFC, another CCD called the High Resolution Channel (HRC), and a near-UV MAMA detector called the Solar Blind Channel (SBC). In January 2007, an electronics failure caused the WFC and HRC detectors to go offline. New electronics were installed in May 2009 during Servicing Mission 4, and while the WFC was successfully recovered, the HRC remains unavailable since the failure. 

The ACS/WFC detector is comprised of two butted CCDs with a 30 pixel gap between them, and the detector provides the largest field of view of any *HST* instrument at 202 x 202 arcseconds. The CCDs are designed to have optimal quantum efficiency (QE) in the middle of the visible spectrum, which complements well the WFC3/UVIS QE that better observes in the near-UV and near-IR. The ACS/WFC is capable of broad-, medium-, and narrow-band imaging, tunable wavelength imaging using ramp filters, wide-field slitless spectroscopy with a grism, and it is the only space-based instrument capable of studying polarized light.

### 1.1 Who Should Use This Notebook?

While this notebook uses the ACS instrument as the basis for an example, any users who are new to *HST* CCD imaging or who are looking to use Python for their analysis will find this information helpful. As always, users are encouraged to review the documentation for their instrument and visit the instrument website for the most recent updates regarding instrument performance and data analysis.

### 1.2 I Need Help!

If you have questions about *HST* data analysis, calibration software, instrument capabilities, and/or the methods discussed in this notebook, please visit the [HST Help Desk](http://hsthelp.stsci.edu). Through the help desk portal, you can explore the *HST* Knowledge Base and request additional help from experts.

### 1.3 Notes

<div class="alert alert-block alert-warning">**Important:** Before proceeding, please be sure to install or update your [AstroConda](https://astroconda.readthedocs.io) distribution. The code cells below may not work properly with older versions of AstroConda.</div>

This example works with data that have been taken with the full ACS/WFC CCD (rather than in a subarray mode). There are additional considerations that must be made for subarray data, which we have noted where appropriate. For more information about ACS/WFC subarrays, please visit the [ACS Website](http://www.stsci.edu/hst/acs).

This example will generate a large volume of data on your disk. You will need approximately 13 GB of space available. At the end of the example is a code block that can be used to remove all of the files except the final calibrated and combined mosaic.

# 2. Download the Data

>Here we will show you how to:
>
>1. Use `astroquery` to download data products from the *HST* archive using a known dataset name

For this example, we have chosen the associated product `J9L960010`, which is an ACS/WFC observation of 47 Tuc (NGC 104) in the F606W filter. The data come from the GO proposal 10775 "An ACS Survey of Galactic Globular Clusters" (PI: A. Sarajedini), and were taken with a series of POS-TARG offsets rather than a pre-defined dither pattern. More information about dithering strategies for each instrument can be found on the appropriate instrument team's website and in the documentation.

Data are downloaded using the `astroquery` API to access the [MAST](http://archive.stsci.edu) archive. The `astroquery.mast` [documentation](http://astroquery.readthedocs.io/en/latest/mast/mast.html) has more examples for how to find and download data from MAST.

In [ ]:
from astroquery.mast import Observations
import shutil
import glob
import os

# Retrieve the observation information.
obs_table = Observations.query_criteria(obs_id='J9L960010')

# Download only the j9l960010_asn.fits, *_raw.fits, and *_spt.fits files.
Observations.download_products(obs_table['obsid'], mrp_only=False, 
                               productSubGroupDescription=['RAW', 'ASN', 'SPT'], 
                               extension='fits')

# Move the files from the mastDownload directory to the current working
# directory and make a backup of the files.
fits_files = glob.glob('mastDownload/HST/J*/*.fits')
for fil in fits_files:
    try:
        shutil.move(fil, '.')
    except:
        os.remove(fil.split('/')[-1])
        shutil.move(fil, '.')
    
# Delete the mastDownload directory and all subdirectories it contains.
shutil.rmtree('mastDownload')

# 3. Calibrate Raw Files

>Here we will show you how to:
>
>1. Query the Calibration Reference Data System ([CRDS](http://www.stsci.edu/hst/observatory/crds/)) for the current best reference files applicable to a given observation
>
>2. Update the `*_raw.fits` primary headers with new calibration information
>
>3. Retrieve calibration files from CRDS and set up the reference file directory
>
>4. Process files with `calacs`

Now that we have the `*_raw.fits` files, we can process them with the ACS calibration pipeline `calacs`. For most observations, reprocessing the raw files with the calibration pipeline is no longer required as the [MAST](http://archive.stsci.edu) archive is now static and any changes to the pipeline or reference files automatically triggers a reprocessing of the data. However, users who wish to apply custom reference files or who have previously downloaded files and wish to update them will find this useful.

### 3.1 Querying CRDS for Reference Files and Updating Headers

Now that the `*_raw.fits` files have all been updated, we need to alter the association file to not create the combined product. We will filter the association file to only include table entries with `MEMTYPE` equal to 'EXP-DTH'. This will remove the 'PROD-DTH' entry that describes the combined product.

In [ ]:
from astropy.io import fits

with fits.open('j9l960010_asn.fits', mode='update') as asn_hdu:
    asn_tab = asn_hdu[1].data
    asn_tab = asn_tab[asn_tab['MEMTYPE'] == 'EXP-DTH']
    asn_hdu[1].data = asn_tab

For this part of the example, we will turn the drizzling step off, as we will perform a custom call to [AstroDrizzle](http://drizzlepac.stsci.edu) at a later stage.

Due to the computationally intense processing required to CTE correct full-frame ACS/WFC images, we have disabled the CTE correction here by default, however it can be turned on by changing the following variable to True:

In [ ]:
cte_correct = False

We also need to set some environment variables for several subsequent calibration tasks:

In [ ]:
import os

os.environ['CRDS_SERVER_URL'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_SERVER'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_PATH'] = './crds_cache'
os.environ['JREF'] = './crds_cache/references/hst/acs/'
os.environ['jref'] = './crds_cache/references/hst/acs/'

Other calibration steps can be enabled or disabled by setting the calibration switch keywords in the primary header to 'PERFORM' or 'OMIT', respectively. All calibration switch keywords can be found in the primary header and all end with the string `CORR` (e.g., `BLEVCORR` and `DARKCORR`). For a complete list of the calibration steps for ACS data, please see the ACS Data Handbook. The *HST* data pipeline determines the default calibration switches for each instrument configuration, therefore the defaults will be sufficient for most users. Some large programs (e.g., Frontier Fields) have performed custom calibration for their science goals. For questions about custom calibration of ACS data, please contact the ACS Branch via the [*HST* Help Desk](http://hsthelp.stsci.edu). Please note that while the ACS Branch answers questions about how ACS data are calibrated using `calacs`, the use of custom calibration pipelines developed outside of the ACS Branch is not supported.

The code block below will query CRDS for the best reference files currently available for these datasets, update the header keywords to point to these new files, and will download the reference files into a subdirectory called "crds_cache/" that we will point to using the JREF environment variable. The "JREF" variable is used for ACS reference files. Other instruments use other variables, e.g., IREF for WFC3.

<div class="alert alert-block alert-warning">**CAUTION:** After Servicing Mission 4 (SM4; May 2009), the installation of the ASIC during the repair of ACS introduced $1/f$ noise in all ACS images. In the calacs pipeline, only full-frame data have this striping removed. To correct subarray data, the alternative acs_destripe_plus pipeline must be used, which will apply all of calibration steps normally performed by calacs in addition to de-striping for subarray data. De-striping is only possible for 2K subarrays after SM4 until Cycle 24, after which a change to the flight software makes all subarrays eligible for de-striping. For more information, see the [ACS website](http://www.stsci.edu/hst/acs).</div>

In [ ]:
from astropy.io import fits
import glob
import os

raw_files = glob.glob('*_raw.fits')

for fil in raw_files:
    with fits.open(fil, mode='update') as hdu:
        if cte_correct:
            hdu[0].header['PCTECORR'] = 'PERFORM'
        else:
            hdu[0].header['PCTECORR'] = 'OMIT'

for fil in raw_files:
    !crds bestrefs --files $fil --sync-references=1 --update-bestrefs

### 3.2 Running calacs

Finally, we can run `calacs` on the association file and it will produce `*_flt.fits` and `*_flc.fits` (if `PCTECORR` = 'PERFORM') files for each "\*\_raw.fits" file. The FLT files have had the normal CCD calibration steps (bias subtraction, dark subtraction, flat field normalization) performed. The FLC files are in all other ways identical to the FLT files, but with the CTE correction also applied.

The `calacs` code is a wrapper around several functions (e.g., `acsccd`, `acs2d`, `acscte`, etc.). While the individual functions may be called through the [acstools](http://acstools.readthedocs.io/en/latest/index.html) package, users are advised to use the `calacs` code. More information about the calibration steps performed by `calacs` can be found in the [ACS Data Handbook](http://www.stsci.edu/hst/acs/documents/handbooks/currentDHB/acs_cover.html) Chapter 3.

<div class="alert alert-block alert-info"><b>Note:</b> If the CTE correction is enabled, this next step will take a long time to complete. The CTE correction is computationally expensive and will use all of the cores on a machine by default. On an 8 core machine, CTE correcting a full-frame ACS/WFC image can take approximately 15 minutes per RAW file. To skip the CTE correction, you can change the `PCTECORR` keyword in the primary headers of the `*_raw.fits` files to 'OMIT' (see above).</div>

In [ ]:
!calacs.e j9l960010_asn.fits

As a check of our calibrated products, we will plot a subsection of one of the input images. Note that the `*_raw.fits` files include bias prescan/overscan regions that are removed as part of the calibration process. ACS/WFC full-frame images have 20 rows of virtual overscan, and 24 columns of physical prescan. Thus, ACS/WFC full-frame `*_raw.fits` files have image sizes of 2068 rows x 4144 columns, while the calibrated FLT and FLC files have dimensions of 2048 rows x 4096 columns.

In [5]:
from ginga.web.pgw import ipg
from code import notebook_tools

gserver = ipg.make_server(host='localhost', port=9916, use_opencv=True)
gserver.start(no_ioloop=True)

if cte_correct:
    img2 = 'j9l960a7q_flc.fits'
else:
    img2 = 'j9l960a7q_flt.fits'

notebook_tools.side2(gserver, 'j9l960a7q_raw.fits', img2)

Comparing the FLT calibrated image to the RAW uncalibrated one, we can see that image artifacts have been removed. Most noticeably, hot columns in the bias have been subtracted. This image was taken prior to SM4, however post-SM4 images would show horizontal stripes in the RAW image, which are removed by `calacs` in the creation of the FLT/FLC images. Checking the pixel values shows that the FLT/FLC image has fewer counts per pixel due to the removal of the bias and dark current. Also note that the image units have changed: RAW ACS images have units of DN, while FLT/FLC images have units of electrons.

The FLT and FLC images are not yet suitable for photometry. Before performing any analysis on the images, we still need to remove detector artifacts, cosmic rays, and geometric distortion. [AstroDrizzle](http://drizzlepac.stsci.edu) can be used to do all of these steps and produce a single mosaic image that incorporates all of the individual exposures.

When combining images with [AstroDrizzle](http://drizzlepac.stsci.edu) for scientific analysis, only images with the same instrument configuration should be combined. One image in the association (`j9l960a7q_raw.fits`) has a different exposure time, therefore we will exclude this dataset from our subsequent analysis.

# 4. Align and Combine Calibrated Images with AstroDrizzle

>In this part, we will show you how to:
>
>1. Update the FLT/FLC file WCS header keywords
>2. Align individual images to a reference image
>3. Combine individual images into a mosaic suitable for photometry

Now that we have created calibrated files from the raw data, we can use [AstroDrizzle](http://drizzlepac.stsci.edu) to astrometrically align our several individual images and create a combined mosaic free of geometric distortion. For detailed information on how the drizzling process works, users are encouraged to read the [AstroDrizzle](http://drizzlepac.stsci.edu) documentation.

### 4.1 Image Alignment

To begin, we need to run `stwcs.updatewcs` on the individual files to update the header keywords to do with the astrometry.

<div class="alert alert-block alert-info"><p><b>Note:</b> The Astrometry Working Group at STScI has recently added absolute astrometry solutions as part of the stwcs.updatewcs functionality. Data processed with calacs v10.0.0 and later will attempt to query AstrometryDB through MAST. The astrometry database is not yet available, which will print a scary warning to the screen. This has no detrimental affect to your data.</p>

<p>In the interim, users interested in improving the absolute astrometry of their observations should consult Varun Bajaj's in-depth guide to astrometric alignment with the GAIA catalog using tweakreg. For more information, see his publication [WFC3 ISR 2017-19](http://www.stsci.edu/hst/wfc3/documents/ISRs/WFC3-2017-19.pdf) and a jupyter notebook at the [gaia_alignment](https://github.com/spacetelescope/gaia_alignment) GitHub repository. More information about the *HST* astrometric accuracy compared to GAIA can be found in [ACS ISR 2018-01](http://www.stsci.edu/hst/acs/documents/isrs/isr1801.pdf) by Kozhurina-Platais, Grogin, & Sabbi.</p></div>

In [ ]:
from stwcs import updatewcs

# Note that the wildcards here will exclude the j9l960a7q dataset.
if cte_correct:
    img_files = 'j9l9*a[9-f]q_flc.fits'
else:
    img_files = 'j9l9*a[9-f]q_flt.fits'

updatewcs.updatewcs(img_files, use_db=False)

The next step is to use `drizzlepac.tweakreg` to improve the alignment between multiple images. This matches unresolved sources in the images to create a transformation for each image relative to a reference image. The offsets computed by `drizzlepac.tweakreg` can be saved to an output file for inspection. Sources used for alignment should be reasonably bright, but not near saturation. Too many sources do not necessarily yield a better transformation, while too few sources make finding a transformation solution difficult. We can run `drizzlepac.tweakreg` to both the FLT and the FLC files (if they were made) in one step.

We have chosen `j9l960abq_flt.fits` to be our reference image, and because we want a reference catalog of well-detected sources, we have set the minimum signal-to-noise for sources to be 4,000. The `dqbits` parameter allows us to only examine pixels with a value of 0 ("good") in the data quality array. We have also placed a restriction that the peak pixel value in a source must be less than 50,000 electrons (above this value, pixels are nearing the full-well saturation). Finally, we have chosen a convolution kernel width of 3.5 pixels, which is appropriate for ACS/WFC and WFC3/UVIS.

In [ ]:
from drizzlepac import tweakreg

tweakreg.TweakReg(img_files, shiftfile=True, interactive=False, 
                  expand_refcat=False, refimage='j9l960abq_flt.fits', 
                  updatehdr=True, runfile='tweak.log',
                  imagefindcfg={'dqbits': 0, 'threshold': 1000, 
                                'peakmax': 50000, 'conv_width': 3.5},
                  refimagefindcfg={'dqbits': 0, 'threshold': 4000, 
                                   'peakmax': 50000, 'conv_width': 3.5})

### 4.2 Combining Images into a Single Mosaic

Now that the relative astrometry has been perfected, we can move on to combining the individual images into a single mosaic. In our call to AstroDrizzle below, we have adjusted some of the default parameters. Let us consider some of the parameters we have adjusted:

* **`driz_sep_bits`** and **`final_bits`**: These control the data quality bit flags that we will consider to be good when correcting individual frames and making the combined mosaic. ACS dark reference files are analyzed for pixel stability (see [ACS ISR 2017-05](http://www.stsci.edu/hst/acs/documents/isrs/isr1705.pdf)), and only *unstable* pixels are considered bad. Data quality flags 16 (hot pixels) and 64 (warm pixels) can be included in our analysis, while flag 32 (unstable pixels) should be excluded. We have also included flag 256 (saturation) in our combined product as pixels that have exceeded the full-well saturation can still be used for photometry (see ACS ISR ????-??). Pixels that exceed the A-to-D saturation (flag 2048) are still excluded. The string we have used in the command below ('16, 64, 256') is equivalent to setting the value to '336', however the former is more transparent to users.


* **`final_rot`**: Setting this to 0 will align our mosaic image such that North is along the positive y-axis and East is along the negative x-axis.


* **`final_wht_type`**: We have selected the exposure time ("EXP") weighting, which is appropriate for use with both Source Extractor and `photutils` photometry. The weight method selection depends strongly on both your data and analysis plan. See the [AstroDrizzle](http://drizzlepac.stsci.edu) documentation for more information.


* **`final_scale`**: This parameter controls the size of the output image pixels and has units of arcseconds per pixel. A value of 0.03 is suitable for ACS/WFC.

In [ ]:
from drizzlepac import astrodrizzle

astrodrizzle.AstroDrizzle(img_files, output='47tuc_f606w_1.0', 
                          driz_sep_bits='16, 64, 256', final_bits='16, 64, 256', 
                          final_wht_type='EXP', final_rot=0, final_scale=0.03, 
                          final_pixfrac=1.0)

### 4.3 Optimizing the final_pixfrac Parameter

We now have a calibrated, cosmic-ray cleaned mosaic free of geometric distortion. However, we can make improve our product using optimal parameters in [AstroDrizzle](http://drizzlepac.stsc.edu) to better sample the ACS/WFC PSF. In particular, we will test several values of the `final_pixfrac` parameter to make a better mosaic.

Note that in this call to [AstroDrizzle](http://drizzlepac.stsci.edu), we have turned off all steps other than the final image combination. We have also taken care to set the `resetbits` option to 0 so that we do not lose the cosmic ray rejection performed as part of the first time we ran AstroDrizzle.

In [ ]:
import numpy

# Create an array of final_pixfrac values ranging from 0.1
# to 0.9 in steps of 0.1.
pixfracs = numpy.linspace(0.1, 0.9, 9)

# Loop over each final_pixfrac value, creating drizzled
# products with root file name "47tuc_f606w_XX" where
# XX is the final_pixfrac value.
for frac in pixfracs:
    astrodrizzle.AstroDrizzle(img_files, 
                              output='47tuc_f606w_' + str(frac), resetbits=0,
                              driz_sep_bits='16, 64, 256', final_bits='16, 64, 256',
                              final_wht_type='EXP', final_rot=0, final_scale=0.03, 
                              final_pixfrac=frac, skysub=False, driz_separate=False,
                              median=False, blot=False, driz_cr=False)

We can now examine all of the drizzled products to identify the one with the best parameters. From the [AstroDrizzle](http://drizzlepac.stsci.edu) documentation, there are three tests to identify the best `final_pixfrac`:

1. Verify that there are not numerous pixels with missing data in the final output
2. Minimize `final_pixfrac` such that the ratio RMS/median is $\leq2$ in the weight map for the area of interest
3. Examine the PSFs of stars and ensure that they are round and not blocky 

From the first test, we can immediately eliminate `final_pixfrac = [0.1, 0.2, 0.3]`. An example of `final_pixfrac = 0.2` is shown below for reference. Pixels with null values are also present along the edges of the mosaic with `final_pixfrac = 0.4`, therefore we will eliminate this value as well. Users are encouraged to examine these images in DS9 to better observe this effect in an interactive way and verify this for themselves.

**Remember, the image shown below is bad:**

In [7]:
from ginga.web.pgw import ipg
from code import notebook_tools
import warnings

warnings.filterwarnings('ignore')

gserver = ipg.make_server(host='localhost', port=9917, use_opencv=True)
gserver.start(no_ioloop=True)

if cte_correct:
    driz_02 = '47tuc_f606w_0.2_drc_sci.fits'
else:
    driz_02 = '47tuc_f606w_0.2_drz_sci.fits'
    
v1 = gserver.get_viewer('v1')
v1.load(driz_02)
v1.embed(height=650, width=650)

Next we can check the value of RMS-to-median ratio in sections of the drizzled weight maps. Here we have selected four boxes in the corners of the mosaic.

In [12]:
from astropy.io import fits
import glob

drz_files = glob.glob('47tuc_f606w_*_drz_wht.fits')

rms_med = []
pixfrac = []

# Now for each drizzled mosaic, compute the RMS/median
# statistic.
for fil in drz_files:
    with fits.open(fil) as hdu:
        box1 = hdu[0].data[2250: 2750, 1000: 1500]
        box2 = hdu[0].data[7250: 7750, 2250: 2750]
        box3 = hdu[0].data[6000: 6500, 7250: 7750]
        box4 = hdu[0].data[1250: 1750, 5750: 6250]
        
        data = np.vstack((box1, box2, box3, box4))
        
        rms_med.append(np.std(data)/np.median(data))
        pixfrac.append(float(fil.split('_')[2]))

Now that we have our measurements from the weight maps, we can plot the results. Here we have hidden the plotting commands in an external function that we import. The plotting is done with [`plotly`](https://plot.ly/) to generate an interactive plot. Interested users can find the plotting code in the acs-notebook repository under `notebooks/code/plots.py`.

In [9]:
from plotly.offline import init_notebook_mode, iplot
from code import plots

init_notebook_mode(connected=True)

figure = plots.drizzle_rms_plot(pixfrac, rms_med)
iplot(figure)

In [8]:
from ginga.web.pgw import ipg
from code import notebook_tools
import warnings

warnings.filterwarnings('ignore')

gserver = ipg.make_server(host='localhost', port=9927, use_opencv=True)
gserver.start(no_ioloop=True)

if cte_correct:
    driz_08 = '47tuc_f606w_0.8_drc_sci.fits'
else:
    driz_08 = '47tuc_f606w_0.8_drz_sci.fits'
    
ystars = [6823, 2937, 1722]
xstars = [7340, 2139, 6129]
    
notebook_tools.side3(gserver, driz_08, driz_08, driz_08, ycen=ystars, xcen=xstars)

In [9]:
from plotly.offline import init_notebook_mode, iplot
from src import plots

ystars = [6823, 2937, 1722]
xstars = [7340, 2139, 6129]

driz_08 = '47tuc_f606w_0.8_drz_sci.fits'

for i, _ in enumerate(ystars):
    if i < 1:
        fig = plots.star_2d(driz_08, ystars[i], xstars[i], size=10, high=150, spacing=20)
    else:
        fig = plots.star_2d(driz_08, ystars[i], xstars[i], size=10, high=500, spacing=70)
    iplot(fig)

# 5. Conclusion

The final image `47tuc_f606w_0.8_drz_sci.fits` (or `47tuc_f606w_0.8_drc_sci.fits` if the CTE correction was performed) is now ready for scientific analysis. All of the necessarily calibration steps have been performed, cosmic rays and geometric distortion have been removed, and the image has units of electrons/second. 

While the calibration process shown here is appropriate for most data, some users may wish to omit certain steps (e.g., the pixel-based CTE correction or drizzling). For users who choose not to use the pixel-based CTE correction implemented in `calacs`, you must apply the [empirical CTE correction](https://acsphotometriccte.stsci.edu/) to your aperture photometry. Users who do not use `astrodrizzle` to correct data for distortion will need to apply a pixel area map to their data to correct for the distorted pixel area projected onto the sky before performing photometry. Instances in which a user would omit the CTE or distortion correction have very specific use cases, and these steps should only be skipped with good justification. More information about these topics can be found in the [ACS Data Handbook]() and on the [ACS website](http://www.stsci.edu/hst/acs).

The final mosaic made at the end of this example is available for download for use in other examples, therefore you do not need to keep a copy of it. To free up disk space on your machine, you can run the code block below to delete the files made as part of this example. If you altered any part of this example, you may need to delete additional files yourself.